# Imports

In [1]:
!pip install pandas


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
from numpy import genfromtxt
import matplotlib.pyplot as plt

In [10]:
# csv_data = pd.read_csv('train.csv')
csv_data = pd.read_csv('/home/daril_kw/data/02.06.23/train_clean.json')


 # Définition des fonctions

In [11]:
def parcourir_couples(polyline) : #polyline de type string
    polyline = polyline[1:(len(polyline)-1)]
    #l'idée est de récupérer les couples en identifiant les " ],[ "
    polyline_n=polyline[1:(len(polyline)-1)] #on enlève les crochets de la liste au complet
    couple = []
    i=0
    while i<len(polyline_n):
        if polyline_n[i]=='[':
            deb_couple = i
            while i<len(polyline_n) and polyline_n[i]!=']':
                i+=1
            couple.append(polyline_n[deb_couple:i+1])
        i+=1
    return couple

#on récupère la liste ci-dessus et on convertit en float chaque nombre qui était stocké en string
def polyline2listcoord(polyline): 
    liste=parcourir_couples(polyline)
    couples =[]
    couples_float = []
    for i in liste :
        couple=i[1:(len(i)-1)]
        couples.append(couple)

        j=0
         #on sépare la longitude et la latitude en identifiant la virgule du couple, puis on recupère avant et après
        while j<len(couple) and couple[j]!=',':
            j+=1
        if couple[j]==',':
            longitude = couple[0:j]
            latitude = couple[j+1:(len(couple))]
        couples_float.append([float(latitude),float(longitude)])
        #attention à l'ordre : h3 demande latitude et longiture dans cet ordre alors que le polyline est l'ensemble 
        #les couples longitude et latitude dans cet ordre
            
    #return liste, couples, couples_float
    return(couples_float)


# Récupération de la colonne et application des fonctions 

In [12]:
poly_list = csv_data['POLYLINE']
polyline2listcoord(poly_list[0])

[[41.141376, -8.618499],
 [41.14251, -8.620326],
 [41.143815, -8.622153],
 [41.144373, -8.623953],
 [41.144778, -8.62668],
 [41.144697, -8.627373],
 [41.14521, -8.630226],
 [41.14692, -8.632746],
 [41.148225, -8.631738],
 [41.150385, -8.629938],
 [41.151213, -8.62911],
 [41.15124, -8.629128],
 [41.152203, -8.628786],
 [41.152374, -8.628687],
 [41.152518, -8.628759],
 [41.15268, -8.630838],
 [41.153022, -8.632323],
 [41.154489, -8.631144],
 [41.154507, -8.630829],
 [41.154516, -8.630829],
 [41.154498, -8.630829],
 [41.15448, -8.630838]]

In [ ]:
csv_data.head()

In [13]:
data_formated = csv_data.apply(lambda x:polyline2listcoord(x.POLYLINE), axis=1)

In [ ]:
type(data_formated)

On modifie le dataset avec la nouvelle colonne.

In [ ]:
csv_data_2 = csv_data
csv_data_2['POLYLINE'] = data_formated

On vérifie le résultat et les formats dans la colonne polyline.

In [ ]:
type(csv_data_2['POLYLINE'][0]),csv_data_2['POLYLINE'][0] 

In [ ]:
type(csv_data_2['POLYLINE'][1][4]), csv_data_2['POLYLINE'][1][4]

In [ ]:
type(csv_data_2['POLYLINE'][2][1][0]), csv_data_2['POLYLINE'][2][1][0]

## Sauvegarde dans un fichier

In [ ]:
csv_data_2.to_csv('formated_trajectories.csv')

## Test de chargement

In [ ]:
csv_data_charge = pd.read_csv('formated_trajectories.csv')

In [ ]:
type(csv_data_charge['POLYLINE'][0])

**Conclusion : c'est le fait d'enregistrer en csv et de le charger qui fait qu'on a des strings au lieu de float**

Une solution pourrait être d'ajouter des colonnes au fichier afin de ne pas stocker des listes de liste et donc espérer pouvoir stocker en float, mais très laborieux (on ne connait pas la longueur des trajectoires a priori etc)

## Tokenization

__On fait la tokenization pour les trajectoires du polyline__

In [ ]:
import h3

In [ ]:
reso=10

def tokenize_list(polyline) :
    for i in range(len(polyline)) :
        polyline[i]=h3.geo_to_h3(polyline[i][0], polyline[i][1], reso)
    return polyline
        

In [ ]:
data_tokenized = csv_data_2.apply(lambda x: tokenize_list(x.POLYLINE), axis=1)

In [ ]:
type(data_tokenized)

In [ ]:
csv_data_token = csv_data
csv_data_token['POLYLINE'] = data_tokenized

In [ ]:
type(csv_data_token['POLYLINE'][0]),csv_data_token['POLYLINE'][0] 

On remplace le polyline par la version tokenizée et on sauvegarde le résultat dans un fichier

In [ ]:
csv_data_token.to_csv('tokenized_trajectories.csv')

In [6]:
csv_data_token_c = pd.read_csv('tokenized_trajectories.csv')

In [7]:
csv_data_token_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1710670 entries, 0 to 1710669
Data columns (total 10 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Unnamed: 0    int64  
 1   TRIP_ID       int64  
 2   CALL_TYPE     object 
 3   ORIGIN_CALL   float64
 4   ORIGIN_STAND  float64
 5   TAXI_ID       int64  
 6   TIMESTAMP     int64  
 7   DAY_TYPE      object 
 8   MISSING_DATA  bool   
 9   POLYLINE      object 
dtypes: bool(1), float64(2), int64(4), object(3)
memory usage: 119.1+ MB


In [8]:
type(csv_data_token_c['POLYLINE'][0]),csv_data_token_c['POLYLINE'][0]

(str,
 "['8a39220f062ffff', '8a39220f0627fff', '8a39220f0707fff', '8a39220f009ffff', '8a39220f0097fff', '8a39220f054ffff', '8a39220f0547fff', '8a39220f0cc7fff', '8a39220f0ceffff', '8a39220f0c4ffff', '8a39220f0137fff', '8a39220f0137fff', '8a39220f0137fff', '8a39220f0127fff', '8a39220f0127fff', '8a39220f0897fff', '8a39220f0d4ffff', '8a39220f08a7fff', '8a39220f08a7fff', '8a39220f08a7fff', '8a39220f08a7fff', '8a39220f08a7fff']")

In [ ]:
h3.geo_to_h3( 41.141376, -8.618499,10)

On a bien la même chose : la tokenization semble avoir fonctionné.

__A présent, on fait de même mais au lieu de remplacer le polyline par sa version tokenizée, on met les deux colonnes.__

In [9]:
csv_data_full = csv_data_token_c
csv_data_full.insert(9, "Trajectory",0,allow_duplicates=False)

In [14]:
csv_data_full['Trajectory']=data_formated

In [15]:
csv_data_full.head()

,Unnamed: 0,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,Trajectory,POLYLINE
0,0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[41.141376, -8.618499], [41.14251, -8.620326]...","['8a39220f062ffff', '8a39220f0627fff', '8a3922..."
1,1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[41.159871, -8.640351], [41.160114, -8.642196...","['8a39220f4757fff', '8a39220f470ffff', '8a3922..."
2,2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[41.14035, -8.613378], [41.140278, -8.614215]...","['8a39220f39a7fff', '8a39220f065ffff', '8a3922..."
3,3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[41.151942, -8.574705], [41.151933, -8.574696...","['8a39220c458ffff', '8a39220c458ffff', '8a3922..."
4,4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[41.180517, -8.645949], [41.180049, -8.646048...","['8a392201b58ffff', '8a392201b58ffff', '8a3922..."


In [16]:
type(csv_data_full['Trajectory'][0]),csv_data_full['Trajectory'][0]

(list,
 [[41.141376, -8.618499],
  [41.14251, -8.620326],
  [41.143815, -8.622153],
  [41.144373, -8.623953],
  [41.144778, -8.62668],
  [41.144697, -8.627373],
  [41.14521, -8.630226],
  [41.14692, -8.632746],
  [41.148225, -8.631738],
  [41.150385, -8.629938],
  [41.151213, -8.62911],
  [41.15124, -8.629128],
  [41.152203, -8.628786],
  [41.152374, -8.628687],
  [41.152518, -8.628759],
  [41.15268, -8.630838],
  [41.153022, -8.632323],
  [41.154489, -8.631144],
  [41.154507, -8.630829],
  [41.154516, -8.630829],
  [41.154498, -8.630829],
  [41.15448, -8.630838]])

Dans la version "résultat direct du calcul", on a bien une liste.

On enregistre le résultat avec les deux dans un fichier : 

In [17]:
csv_data_full.to_csv('dataset_traj&token.csv')

In [18]:
csv_test_load_all=pd.read_csv('dataset_traj&token.csv')

In [19]:
type(csv_test_load_all['Trajectory'][0]),csv_test_load_all['Trajectory'][0]

(str,
 '[[41.141376, -8.618499], [41.14251, -8.620326], [41.143815, -8.622153], [41.144373, -8.623953], [41.144778, -8.62668], [41.144697, -8.627373], [41.14521, -8.630226], [41.14692, -8.632746], [41.148225, -8.631738], [41.150385, -8.629938], [41.151213, -8.62911], [41.15124, -8.629128], [41.152203, -8.628786], [41.152374, -8.628687], [41.152518, -8.628759], [41.15268, -8.630838], [41.153022, -8.632323], [41.154489, -8.631144], [41.154507, -8.630829], [41.154516, -8.630829], [41.154498, -8.630829], [41.15448, -8.630838]]')

Dans la version chargée, on se retrouve à nouveau avec une chaîne de caractères. 

In [20]:
csv_test_load_all.head()

,Unnamed: 0.1,Unnamed: 0,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,Trajectory,POLYLINE
0,0,0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[41.141376, -8.618499], [41.14251, -8.620326]...","['8a39220f062ffff', '8a39220f0627fff', '8a3922..."
1,1,1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[41.159871, -8.640351], [41.160114, -8.642196...","['8a39220f4757fff', '8a39220f470ffff', '8a3922..."
2,2,2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[41.14035, -8.613378], [41.140278, -8.614215]...","['8a39220f39a7fff', '8a39220f065ffff', '8a3922..."
3,3,3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[41.151942, -8.574705], [41.151933, -8.574696...","['8a39220c458ffff', '8a39220c458ffff', '8a3922..."
4,4,4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[41.180517, -8.645949], [41.180049, -8.646048...","['8a392201b58ffff', '8a392201b58ffff', '8a3922..."


__Remarque :__ on a toujours le même problème de string. Que ce soit pour les tokens ou les points géographiques, on n'a pas des listes (de chaînes de caractères ou de floats) mais une seule gramde chaîn de caractère.